# Von TEI-XML zu Volltext und Tokenisierung

In diesem Abschnitt werden wir die im vorherigen Unterkapitel gesammelten Daten nutzen, um die Volltexte der Briefe in unseren Dataframe zu laden. Hier nochmal der Hinweis, dass wir eine Auswahl getroffen haben: Wir werden nur die Briefe herunterladen, die in der [edition humboldt digital](https://edition-humboldt.de/) enthalten sind und die eine genaue Datierung besitzen. Da wir über die Anfrage die Briefe im TEI-XML Format erhalten, wir aber nur mit den Volltexten arbeiten wollen, ergibt sich daraus für uns das Ziel, die xml-tags zu entfernen. Zunächst aber laden wir die benötigten Python-Pakete und die Daten, die wir in einer csv-Datei gespeichert haben. 

## Import und Einlesen der Daten

In [59]:
import string
import requests

import pandas as pd

from bs4 import BeautifulSoup as bs

Beim Einlesen der Daten wird mit dem Argument `parse_date=` die Spalte mit Datumsangaben in ein Datetime-Objekt umgewandelt. Dies ermöglicht weitere Optionen für das Arbeiten mit Zeitangaben.

In [60]:
df = pd.read_csv('240228-AvH-letters-with-date.csv', parse_dates=['date'])
print(df.shape)
df.head()

(296, 9)


,reference,edition_id,sender_id,sender,receiver_id,receiver,date,place_id,place
0,https://edition-humboldt.de/H0002655,#AVHR,http://d-nb.info/gnd/118554700,Alexander von Humboldt,http://d-nb.info/gnd/118805193,Samuel Thomas von Soemmerring,1793-12-05,http://sws.geonames.org/6556797,Berg
1,https://edition-humboldt.de/H0002730,#AVHR,http://d-nb.info/gnd/118554700,Alexander von Humboldt,http://d-nb.info/gnd/118805193,Samuel Thomas von Soemmerring,1794-02-06,http://sws.geonames.org/2951825,Bayreuth
2,https://edition-humboldt.de/H0002729,#AVHR,http://d-nb.info/gnd/118554700,Alexander von Humboldt,http://d-nb.info/gnd/118805193,Samuel Thomas von Soemmerring,1795-06-07,http://sws.geonames.org/2951825,Bayreuth
3,https://edition-humboldt.de/H0002657,#AVHR,http://d-nb.info/gnd/118554700,Alexander von Humboldt,http://d-nb.info/gnd/118805193,Samuel Thomas von Soemmerring,1795-06-29,http://sws.geonames.org/2919290,Goldkronach
4,https://edition-humboldt.de/H0001183,#AVHR,http://d-nb.info/gnd/118554700,Alexander von Humboldt,http://d-nb.info/gnd/117387436,Karl Ludwig Willdenow,1795-07-17,http://sws.geonames.org/2951825,Bayreuth


## Definieren von Funktionen

Pandas bietet den Vorteil, einzelne Spalten eines Dataframes, Series, zu bearbeiten. Hierzu können die nötigen Funktionen entsprechend definiert und mit `apply()` angewandt werden. `apply()` ist wesentlich perfomanter als for-Schleifen, die hier ja eigentlich auch in Frage kommen könnten. 

Die folgenden Funktionen nutzen wir später, um die TEI-XML Dateien herunterzuladen, den reinen Text aus dem XML-Format herauszuziehen, um bei dem erhaltenen Text die Zeichensetzung zu entfernen und um diesen in Token zu zerlegen. Die Doc-Strings zu den Funktionen geben zusätzlich kurze Infos dazu, was die jeweilige Funktion ausführt.

In [61]:
def get_xml_file(reference):
    '''
    Function makes URL-request.
    Concatenates string and uses requests to get file.

    Return: requests-Response-Object    
    '''
    response = requests.get(reference + '.xml')
 
    return response

In [62]:
def get_text(reference):
    '''
    Function returns from xml-tags cleaned text.
    Calls helperfunction and makes soup to parse xml.
    Finds relevant part of xml and strips xml-tags.
    
    Return: string
    '''
    response = get_xml_file(reference)

    soup = bs(response.text, 'html.parser')

    text = soup.find('text') 

    clean_text = ' '.join(bs(str(text), 'html.parser').stripped_strings)

    return clean_text

In [63]:
def get_token(text):
    '''
    Function returns tokens from text.
    Removes whitespaces and punctuation.
    Normalizes tokens to lowercase.    
    
    Returns: list    
    '''
    text = ' '.join(text.split())

    for char in text:

        if char in string.punctuation + '—':
            text = text.replace(char, '')

    tokens = text.split()

    tokens = [ token.lower() for token in tokens ]

    return tokens

## Aufruf der Funktionen

In der nachfolgenden Notebookzelle rufen wir die Funktionen mit `apply()` jeweils auf einer Spalten auf. Die Rückgaben werden direkt neuen Spalten zugewiesen. Wie bei allen Benennungen ist es auch hier empfehlenswert, sprechende Titel für die Spalten zu nutzen. Die Ausführung der Zelle kann etwa 1 Minute dauern, da die Anfragen an einen externen Server gestellt werden. Dann geben wir die Anzahl von Spalten und Zeilen sowie die ersten Zeilen aus, um das Ergebnis zu prüfen. 

In [64]:
df.loc[:, 'text'] = df.loc[:, 'reference'].apply(get_text)
df.loc[:, 'token'] = df.loc[:, 'text'].apply(get_token)
df.loc[:, 'nr_token'] = df.loc[:, 'token'].apply(len)

print(df.shape)
df.head()

(296, 12)


,reference,edition_id,sender_id,sender,receiver_id,receiver,date,place_id,place,text,token,nr_token
0,https://edition-humboldt.de/H0002655,#AVHR,http://d-nb.info/gnd/118554700,Alexander von Humboldt,http://d-nb.info/gnd/118805193,Samuel Thomas von Soemmerring,1793-12-05,http://sws.geonames.org/6556797,Berg,"Berg , den 5 . Dez Dezember 1793 Ihr Brief, li...","[berg, den, 5, dez, dezember, 1793, ihr, brief...",252
1,https://edition-humboldt.de/H0002730,#AVHR,http://d-nb.info/gnd/118554700,Alexander von Humboldt,http://d-nb.info/gnd/118805193,Samuel Thomas von Soemmerring,1794-02-06,http://sws.geonames.org/2951825,Bayreuth,"Der Ueberbringer dieses Briefchens, lieber Soe...","[der, ueberbringer, dieses, briefchens, lieber...",143
2,https://edition-humboldt.de/H0002729,#AVHR,http://d-nb.info/gnd/118554700,Alexander von Humboldt,http://d-nb.info/gnd/118805193,Samuel Thomas von Soemmerring,1795-06-07,http://sws.geonames.org/2951825,Bayreuth,"Verehrungswerther Freund, Ich will Ihnen ein B...","[verehrungswerther, freund, ich, will, ihnen, ...",591
3,https://edition-humboldt.de/H0002657,#AVHR,http://d-nb.info/gnd/118554700,Alexander von Humboldt,http://d-nb.info/gnd/118805193,Samuel Thomas von Soemmerring,1795-06-29,http://sws.geonames.org/2919290,Goldkronach,"Wie innig, theurer, Verehrungswerther Mann! fr...","[wie, innig, theurer, verehrungswerther, mann,...",899
4,https://edition-humboldt.de/H0001183,#AVHR,http://d-nb.info/gnd/118554700,Alexander von Humboldt,http://d-nb.info/gnd/117387436,Karl Ludwig Willdenow,1795-07-17,http://sws.geonames.org/2951825,Bayreuth,"ich reise noch heute von hier nach Venedig , d...","[ich, reise, noch, heute, von, hier, nach, ven...",143


## Speichern der Ergebnisse

Abschließend speichern wir die Ergebnisse. Diesmal nutzen wir das csv- und das JSON-Format. Der Vorteil von csv ist, das wir dieses Format auch leicht mit anderen Tabellenkalkulationsprogrammen öffnen können. Allerdings bleiben die Datentypen in den Zellen nicht erhalten - wir können beim Einlesen von csv-Dateien nur mit Strings weiterarbeiten und müssten diese dann konvertieren. JSON bietet hingegen den Vorteil, dass die Datentypen in den jeweiligen Spalten erhalten bleiben: Sind Listen oder Dictionaries in einer Spalten, dann stehen diese Datentypen auch nach dem erneuten Einlesen der JSON-Datei wieder direkt zur Verfügung. Lediglich Spalten, die Datetime-Objekte enthalten, müssten von der unix-Zeit wieder in eine Datetime-Objekt umgewandelt werden. Aber das schauen wir uns im nächsten Abschnitt genauer an.

In [65]:
df.to_csv('240301-AvH-letters-with-tokens.csv', encoding='utf8', index=False)

In [66]:
df.to_json('240301-AvH-letters-with-tokens.json')